<a href="https://colab.research.google.com/github/lookmeebbear/OpenRS_LDDTNI/blob/main/PythonGEE_Rural_LandcoverClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automatic Land Cover Classification by Google Earth Engine**
# **Rural Area -- Prachub Kirikhan**

Thepchai Srinoi

Department of Survey Engineering

Faculty of Engineering Chulalongkorn University 

-------------------------------------------------------------

(1) Install geemap for convert from pandas to gee

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=8d4425f49bd1b0ecae0169064768a018b62885c21e9e544dd7f689

In [2]:
import geemap
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from mlxtend.plotting import plot_confusion_matrix

(2) Retrieve the satellite image from google earth engine

In [3]:
import ee
# Authenticate to the Earth Engine servers
ee.Authenticate()
# Initialize the API
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ZQnhDocLQpoBwRKAGM9nlOQZ9MQGelafn1dGmItGbNo&tc=CnpybCUQrXqmJiQrpDHKOK0w_f-dT06_1V1YjSs119k&cc=LN3stIPgTXm8rlxGE9JjZN5j2xqjApNjFeFFtzQxzHI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VMyctC5XblVQpmDOc857b06LGHEEgVLJsAcUQ0l9ec3ju36-0UANhY

Successfully saved authorization token.


In [12]:
import folium
# Install Function for Cloud Mask 
def applyScaleFactors(image) :
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

def maskS2clouds(image) :
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloudBitMask).eq(0)) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Map Visualization : Google Earth Engine
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

In [39]:
# Retrieve satellite image
lat_min, lat_max, lon_min, lon_max = (12.1256, 12.8229, 99.2070, 99.9018)
AOI = ee.Geometry.Polygon(
        [[[lon_min, lat_max],
           [lon_min, lat_min],
           [lon_max, lat_min],
           [lon_max, lat_max]]])


myimage = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
            .filterDate('2018-1-1', '2018-3-1')\
            .filterBounds(AOI)\
            .filterMetadata('CLOUD_COVER','less_than',5)\
            .map(applyScaleFactors)\
            .mosaic()
            
#True Composite Visualization
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=13)
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.4, 'bands':['SR_B4', 'SR_B3', 'SR_B2']},'myimage_truecolor')
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.4, 'bands':['SR_B5', 'SR_B6', 'SR_B4']},'myimage_falsecolor')
display(map_matched.add_child(folium.LayerControl()))

(3) Download the sample dataset (train, test dataset)

0 - water

1 - rubber

2 - fruit

3 - forest
 
4 - man made land 

In [54]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/lookmeebbear/OpenRS_LDDTNI/main/ldd_samplepoint.csv')
df = df.loc[(df.X > 0) & (df.Y > 0)]
df

,X,Y,id,class
0,99.505763,12.528545,NaN,0
1,99.524514,12.528218,NaN,0
2,99.532427,12.528936,NaN,0
3,99.559481,12.548349,NaN,0
4,99.527552,12.550599,NaN,0
...,...,...,...,...
95,99.587187,12.487781,NaN,4
96,99.543298,12.530148,NaN,4
97,99.547958,12.529852,NaN,4
98,99.552143,12.548424,NaN,4


In [55]:
# Map Visualization : Sampling Point 
classid = list( set(df['class'].to_list()) )
colorid = ['blue','orange','purple','green','red']
k = -1
for myid in classid :
  k += 1
  df_select = df.loc[ df['class'] == classid[myid] ].copy()
  latitudes = list(df_select.Y)
  longitudes = list(df_select.X)
  for lat, lng in zip(latitudes, longitudes):
    folium.CircleMarker(location = [lat, lng], radius=3, color= colorid[k]).add_to(map_matched)

display(map_matched.add_child(folium.LayerControl()))

(4) Let's do supervised classification ...

In [56]:
df = df.drop(['id'],axis=1)
df

,X,Y,class
0,99.505763,12.528545,0
1,99.524514,12.528218,0
2,99.532427,12.528936,0
3,99.559481,12.548349,0
4,99.527552,12.550599,0
...,...,...,...
95,99.587187,12.487781,4
96,99.543298,12.530148,4
97,99.547958,12.529852,4
98,99.552143,12.548424,4


In [71]:
from sklearn.model_selection import train_test_split

X = df[df.columns[:2]]
Y = df[df.columns[2]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.70, random_state = 25)

df_train = X_train.join(Y_train)
df_test = X_test.join(Y_test)

In [46]:
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
#Convert from pandas to featurecollection
train_points = geemap.pandas_to_ee(df_train, latitude='Y', longitude='X')
test_points = geemap.pandas_to_ee(df_test, latitude='Y', longitude='X')

In [119]:
###### CLASSIFICATION #####

#Choose bands
bands = ['SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']

# Overlay the points on the imagery to get training.
training = myimage.sampleRegions(
  collection=train_points,
  properties=['class'],
  scale=30)

# Train a RF,SVM classifier with default parameters.
trained_RF = ee.Classifier.smileRandomForest(5).train(training, 'class', bands)

# Classify the image with the same bands used for training.
classified_RF = (myimage.select(bands)).classify(trained_RF)

# Accuracy Ascessment
testing = myimage.sampleRegions(collection=test_points, properties=['class'], scale=30) #get raster value first !!!!
testResults_RF = (testing.classify(trained_RF)) #find answer from train actor
testAccuracy_RF = (testResults_RF).errorMatrix('class', 'classification') 

print('Random Forest')
print('Validation error matrix: ', testAccuracy_RF.getInfo())
print('Validation error matrix')
for line in testAccuracy_RF.getInfo() : print(line)
print('Validation overall accuracy: ', testAccuracy_RF.accuracy().getInfo())


Random Forest
Validation error matrix:  [[9, 3, 2, 0, 0], [0, 9, 1, 3, 1], [1, 0, 12, 1, 0], [0, 1, 1, 12, 0], [0, 0, 0, 0, 14]]
Validation error matrix
[9, 3, 2, 0, 0]
[0, 9, 1, 3, 1]
[1, 0, 12, 1, 0]
[0, 1, 1, 12, 0]
[0, 0, 0, 0, 14]
Validation overall accuracy:  0.8


Creates a confusion matrix. Axis 0 (the rows) of the matrix correspond to the actual values, and Axis 1 (the columns) to the predicted values.

In [87]:
#Visualization
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=12)
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.3, 'bands':['SR_B4', 'SR_B3', 'SR_B2']},'myimage_truecolor')
map_matched.add_ee_layer(classified_RF,{'min':0, 'max':4, 'palette':colorid},'myimage_classy_RF')
display(map_matched.add_child(folium.LayerControl()))

In [113]:
###### CLASSIFICATION #####

#Choose bands
bands = ['SR_B5','SR_B6']

# Overlay the points on the imagery to get training.
training = myimage.sampleRegions(
  collection=train_points,
  properties=['class'],
  scale=10)

# Train a SVM classifier
trained_SVM = ee.Classifier.libsvm("Voting","C_SVC","SIGMOID")\
                          .train(training, 'class', bands)

# Classify the image 
classified_SVM = (myimage.select(bands)).classify(trained_SVM)

# Accuracy Ascessment
testing = myimage.sampleRegions(collection=test_points, properties=['class'], scale=10) #get raster value first !!!!
testResults_SVM = (testing.classify(trained_SVM)) #find answer from train actor
testAccuracy_SVM = (testResults_SVM).errorMatrix('class', 'classification') 

print('Support Vector Machine')
print('Validation error matrix: ', testAccuracy_SVM.getInfo())
print('Validation error matrix')
for line in testAccuracy_SVM.getInfo() : print(line)
print('Validation overall accuracy: ', testAccuracy_SVM.accuracy().getInfo())



Support Vector Machine
Validation error matrix:  [[12, 1, 0, 1, 0], [0, 14, 0, 0, 0], [0, 2, 12, 0, 0], [3, 6, 0, 5, 0], [0, 14, 0, 0, 0]]
Validation error matrix
[12, 1, 0, 1, 0]
[0, 14, 0, 0, 0]
[0, 2, 12, 0, 0]
[3, 6, 0, 5, 0]
[0, 14, 0, 0, 0]
Validation overall accuracy:  0.6142857142857143


In [105]:
#Visualization
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=12)
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.3, 'bands':['SR_B4', 'SR_B3', 'SR_B2']},'myimage_truecolor')
map_matched.add_ee_layer(classified_RF,{'min':0, 'max':4, 'palette':colorid},'myimage_classy_RF')
map_matched.add_ee_layer(classified_SVM,{'min':0, 'max':4, 'palette':colorid},'myimage_classy_SVM')
display(map_matched.add_child(folium.LayerControl()))

In [114]:
###### CLASSIFICATION #####

#Choose bands
bands = ['SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']

# Overlay the points on the imagery to get training.
training = myimage.sampleRegions(
  collection=train_points,
  properties=['class'],
  scale=10)

# Train a SVM classifier
trained_C = ee.Classifier.smileCart().train(training, 'class', bands)

# Classify the image 
classified_C = (myimage.select(bands)).classify(trained_C)

# Accuracy Ascessment
testing = myimage.sampleRegions(collection=test_points, properties=['class'], scale=10) #get raster value first !!!!
testResults_C = (testing.classify(trained_C)) #find answer from train actor
testAccuracy_C = (testResults_C).errorMatrix('class', 'classification') 

print('CART')
print('Validation error matrix: ', testAccuracy_C.getInfo())
print('Validation error matrix')
for line in testAccuracy_C.getInfo() : print(line)
print('Validation overall accuracy: ', testAccuracy_C.accuracy().getInfo())


CART
Validation error matrix:  [[12, 0, 1, 0, 1], [0, 10, 0, 3, 1], [0, 0, 13, 1, 0], [2, 2, 0, 10, 0], [0, 0, 2, 0, 12]]
Validation error matrix
[12, 0, 1, 0, 1]
[0, 10, 0, 3, 1]
[0, 0, 13, 1, 0]
[2, 2, 0, 10, 0]
[0, 0, 2, 0, 12]
Validation overall accuracy:  0.8142857142857143


In [117]:
#Visualization
folium.Map.add_ee_layer = add_ee_layer
c = (AOI.centroid().getInfo())['coordinates']
map_matched = folium.Map(location=[c[1], c[0]], zoom_start=12)
map_matched.add_ee_layer(myimage,{'min':0, 'max':0.3, 'bands':['SR_B4', 'SR_B3', 'SR_B2']},'myimage_truecolor')
map_matched.add_ee_layer(classified_RF,{'min':0, 'max':4, 'palette':colorid},'myimage_classy_RF')
map_matched.add_ee_layer(classified_C,{'min':0, 'max':4, 'palette':colorid},'myimage_classy_CART')
display(map_matched.add_child(folium.LayerControl()))

# **THE END**

31 Mar 2022 .... Thepchai Srinoi

Update for LDD Training 6 May 2023 ... Thepchai Srinoi